In [1]:
import wolf
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import re
import subprocess

#tasks.py
CLUMPS_DOCKER_IMAGE = "gcr.io/broad-getzlab-workflows/clumps:v55"

class clumps_prep_task(wolf.Task):
    # Preparation for clumps input files:
    # computes mutational frequencies, spectra, and identifies protein structures needed
    resources = { "mem" : "8G" }
    
    # input data for the 'prep' step is the mutation annotation file (maf)
    # <Required> Input file for CLUMPS. Default expects .maf
    inputs = {
        "maf" : None,
        "genome_2bit" : None,
        "fasta" : None, 
        "gpmaps" : None
    }

    script = """
    mkdir clumps_preprocess
    clumps-prep --input ${maf} --output_dir clumps_preprocess --hgfile ${genome_2bit} --fasta ${fasta} --gpmaps ${gpmaps}
    """

    output_patterns = {
        "prep_outdir" : "clumps_preprocess"
    }
    
    docker = CLUMPS_DOCKER_IMAGE

class clumps_run_task(wolf.Task):
    # this task is the main clumps processing/algorithm
    resources = { "mem" : "8G" }
    conf = {"clust_frac": 0.99}
    # the input files for this step are the different individual prot2pdb chunks from the huniprot2pdb_chunks folder
    # provide a list of all the individual prot2pdb chunks (or the file path to each prot2pdb chunks file)
    
    # <Required> Directory of files titled with Uniprot IDs that have mutation information
    # <Required> File mapping uniprot ID to PDB ID with residue-level mapping information.
    # coverage_track is on the gs bucket
    inputs = {
        "clumps_preprocess" : None,
        "prot2pdb_chunks" : None,
        "pdb_dir" : None,
        "coverage_track" : None,
        "coverage_track_index" : None, # not actually used as an input; just needs to be localized alongside coverage_track
        "genome_2bit" : None,
        "fasta" : None,
        "gpmaps" : None,
        "sampler" : "UniformSampler",
        "max_perms" : 10000
    }

    overrides = { "prot2pdb_chunks" : "delayed" }

    script = """    
    clumps --muts ${clumps_preprocess}/split_proteins \
        --maps ${prot2pdb_chunks} \
        --mut_freq ${clumps_preprocess}/mut_freq.txt \
        --out_dir clumps_results \
        --sampler ${sampler} \
        --coverage_track ${coverage_track} \
        --mut_spectra ${clumps_preprocess}/mut_spectra.txt \
        --pdb_dir ${pdb_dir} \
        --hgfile ${genome_2bit} --fasta ${fasta} --gpmaps ${gpmaps} \
        --max_rand ${max_perms} \
        --threads 8
    """

    resources = { "cpus-per-task" : 8 }

    output_patterns = {
        "run_outdir" : "clumps_results"
    }
    
    docker = CLUMPS_DOCKER_IMAGE

class clumps_postprocess_task(wolf.Task):
    # Generates summary files from array outputs of clumps.
    resources = { "mem" : "8G" }
    
    inputs = {
        "clumps_preprocess" : None,
        "clumps_results" : None, # this is an array of directories, which in turn contain multiple files
        "cancer_genes" : None,
        "uniprot_map" : None,
        "pdb_dir" : None
    }
    
    script = """
    # gather contents of all clumps results directories into single directory
    mkdir results_gather
    cat ${clumps_results} | xargs -I{} find -L {} -type f -not -path "*/.*" | xargs -P100 -I{} ln -s {} results_gather
    clumps-postprocess --input_dir results_gather \
      --proteins_dir ${clumps_preprocess}/split_proteins \
      --cancer_genes ${cancer_genes} \
      --uniprot_map ${uniprot_map} \
      --pdb_dir ${pdb_dir} \
      --output_file clumps_output.tsv
    """
    
    # Output file from CLUMPS with list of genes
    output_patterns = {
        "clumps_output" : "clumps_output.tsv"
    }
    
    # Docker Image
    docker = CLUMPS_DOCKER_IMAGE

###### workflow
def clumps_workflow(
  maf,
  sampler,
  genome_2bit = "gs://sa-clumps2-ref/dat/hg19.2bit",
  fasta = "gs://sa-clumps2-ref/dat/UP000005640_9606.fasta.gz",
  gpmaps = "gs://sa-clumps2-ref/dat/genomeProteomeMaps.txt",
  prot2pdb_chunks = "gs://sa-clumps2-ref/dat/huniprot/huniprot2pdb.run18_chunks/",
  pdb_dir = "gs://sa-clumps2-ref/dat/pdbs/ftp.wwpdb.org/pub/pdb/data/structures/divided/pdb",
  coverage_track = "gs://sa-clumps2-ref/dat/cov/WEx_cov.fwb", # TODO: need to specify index here too, and add it to the localization task
  cancer_genes = "gs://sa-clumps2-ref/dat/allCancerGenes.txt",
  uniprot_map = "gs://sa-clumps2-ref/dat/huniprot/huniprot2pdb.run18.filt.txt"
):
    # localization task
    localization = wolf.LocalizeToDisk(
      files = {
        "maf" : maf,
        "genome_2bit" : genome_2bit,
        "fasta" : fasta,
        "gpmaps" : gpmaps,
        "prot2pdb_chunks" : prot2pdb_chunks, # XXX: do we need this on the RODISK? we are scattering directly from the bucket
        "pdb_dir" : pdb_dir,
        "coverage_track" : coverage_track,
        "cancer_genes" : cancer_genes,
        "uniprot_map" : uniprot_map
      }
    )

    # map genome to proteome
    clumps_prep = clumps_prep_task(
      inputs = {
        "maf" : localization["maf"],
        "genome_2bit" : localization["genome_2bit"],
        "fasta" : localization["fasta"],
        "gpmaps" : localization["gpmaps"]
      }
    )

    # list chunks to define scatter
    # (since we can't natively scatter against a directory)
    chunk_list = wolf.Task(
      name = "list_chunks",
      inputs = { "chunks" : prot2pdb_chunks },
      overrides = { "chunks" : "string" },
      script = "gsutil ls ${chunks} > chunks.txt",
      outputs = { "chunks" : ("chunks.txt", wolf.read_lines) }
    )

    # run clumps on each shard
    clumps_run = clumps_run_task(
      inputs = {
        "clumps_preprocess" : clumps_prep["prep_outdir"],
        "prot2pdb_chunks" : chunk_list["chunks"],
        "pdb_dir" : localization["pdb_dir"],
        "coverage_track" : localization["coverage_track"],
        "coverage_track_index" : coverage_track[:-4] + ".fwi", # FIXME: hack to avoid recreating the rodisk
        "genome_2bit" : localization["genome_2bit"],
        "fasta" : localization["fasta"],
        "gpmaps" : localization["gpmaps"],
        "sampler" : sampler
      }
    )

    clumps_postprocess = clumps_postprocess_task(
      inputs = {
        "clumps_preprocess" : clumps_prep["prep_outdir"],
        "clumps_results" : [clumps_run["run_outdir"]],
        "cancer_genes" : localization["cancer_genes"],
        "uniprot_map" : localization["uniprot_map"],
        "pdb_dir" : localization["pdb_dir"]
      }
    )

### CLUMPS
#### - `GitHub repo`: https://github.com/broadinstitute/getzlab-CLUMPS2
---

#### 3 main tasks from the `CLUMPS` algorithm
- _(task 0) **localization** : localization_
- **prep** : pre-processing
- **run** : core clumps algorithm
- **post** : post-processing
---

### (0) LOCALIZATION task

In [2]:
#adding af_dir, prot2af_chunks
clumpsLocalization_results = {'cancer_genes': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/cancer_genes/allCancerGenes.txt', 
                              'coverage_track': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/coverage_track/WEx_cov.fwb', 
                              'coverage_track_index': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/coverage_track_index/WEx_cov.fwi', 
                              'fasta': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/fasta/UP000005640_9606.fasta.gz', 
                              'genome_2bit': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/genome_2bit/hg19.2bit',  
                              'gpmaps': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/gpmaps/genomeProteomeMaps.txt', 
                              'maf': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/maf/pancan15k.v18.maf', 
                              'pdb_dir': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/pdb_dir/pdb', 
                              'af_dir' : 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/af_dir',
                              'prot2pdb_chunks': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/prot2pdb_chunks/huniprot2pdb.run18_chunks', 
                              'prot2af_chunks': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks',
                              'uniprot_map': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/uniprot_map/huniprot2pdb.run18.filt.txt', 
                              'uniprot2af_map': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/uniprot2af_map',
                              'small_maf': 'rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd/small_maf/pancan100.test.maf'}


#docker_image = 'gcr.io/broad-getzlab-workflows/adunford_clumps_highertimeout:latest'
run_name = "clumps_full_AF"


### (1) PREP task

In [ ]:
clumpsPrep = clumps_prep_task()
clumpsPrep_results = clumpsPrep.run(
    maf = clumpsLocalization_results["maf"],
    genome_2bit = clumpsLocalization_results["genome_2bit"],
    fasta = clumpsLocalization_results["fasta"],
    gpmaps = clumpsLocalization_results["gpmaps"]
)

[20220829-21:07:47] [prefect] Available disk storage at /mnt/nfs is small (154 GB remaining)
[20220829-21:07:47] [prefect] Starting Slurm controller ...
[20220829-21:07:47] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20220829-21:07:58] [prefect] Localizing inputs...
[20220829-21:07:58] [prefect] Job staged on SLURM controller in: /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c
[20220829-21:07:58] [prefect] Preparing pipeline script


---

In [6]:
#pick the little chunk list I made.  May need to be a list.
prot2af_dir = clumpsLocalization_results['prot2af_chunks'].replace('rodisk://canine-b7f9872002626d1a514fcdf5e295a8bd','')
chunks_list = subprocess.check_output("ls %s/*" %prot2af_dir, shell = True).decode().rstrip().split("\n")
chunks_list

['/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00000.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00001.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00002.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00003.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00004.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00005.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00006.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00007.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00008.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/prot2AF_chunks/prot2af_00009.gz',
 '/mnt/nfs/ro_disks/canine-b7f9872002626d1a514fcdf5e295a8bd/

### (2) RUN task

In [21]:

clumpsRun = clumps_run_task()
clumpsRun_results = clumpsRun.run(
    clumps_preprocess = clumpsPrep_results["prep_outdir"],
    prot2pdb_chunks = chunks_list,
    pdb_dir = clumpsLocalization_results["af_dir"],
    coverage_track = clumpsLocalization_results["coverage_track"],
    coverage_track_index = clumpsLocalization_results["coverage_track_index"],
    genome_2bit = clumpsLocalization_results["genome_2bit"],
    fasta = clumpsLocalization_results["fasta"],
    gpmaps = clumpsLocalization_results["gpmaps"],
    sampler = 'UniformSampler',
    max_perms = 10000
)

[20220830-17:22:10] [prefect] Available disk storage at /mnt/nfs is small (153 GB remaining)
[20220830-17:22:10] [prefect] Starting Slurm controller ...
[20220830-17:22:10] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/mut_spectra.txt (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/muts.gp (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess; 100/0 MiB completed
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_protein

[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/O43603 (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q6S9Z5 (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9Y4E8 (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q6IPR3 (0 MiB)
[20220830-17:22:11] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8N1E6 (0 

[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P31689 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9BR77 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q96SY0 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9H3Y8 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q969T3 (0 

[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P11940 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q7Z442 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9ULJ6 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8NGH8 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9H6T0 (0 

[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P49763 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/O15226 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8TE85 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q02577 (0 MiB)
[20220830-17:22:12] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P08236 (0 

...

[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/O43790 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P0CF51 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8WW35 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q96EB1 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8TF66 (0 

[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8NFZ8 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9UM01 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P61925 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P05455 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/O94868 (0 

[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P26436 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q96E11 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/P22680 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9P0K9 (0 MiB)
[20220830-17:22:41] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8WZ84 (0 

[20220830-17:22:42] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q92572 (0 MiB)
[20220830-17:22:42] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q15785 (0 MiB)
[20220830-17:22:42] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q8WU43 (0 MiB)
[20220830-17:22:42] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q99456 (0 MiB)
[20220830-17:22:42] [prefect] Hashing file /mnt/nfs/workspace/clumps_prep_task__2022-08-29--20-55-13_5c3fg4i_2vzaanq_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess/split_proteins/Q9UBE8 (0 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### (3) POST-PROCESS task

In [26]:
# THIS IS FOR TESTING
# running the `post_task`
clumpsPost = clumps_postprocess_task()
clumpsPost_results = clumpsPost.run(
    clumps_preprocess = clumpsPrep_results["prep_outdir"],
    clumps_results = [clumpsRun_results["run_outdir"]],
    cancer_genes = clumpsLocalization_results["cancer_genes"],
    uniprot_map = chunks_list,
    #uniprot_map = clumpsLocalization_results["prot2af_chunks"],
    pdb_dir = clumpsLocalization_results["af_dir"]
)

[20220829-17:04:03] [prefect] Available disk storage at /mnt/nfs is small (146 GB remaining)
[20220829-17:04:03] [prefect] Starting Slurm controller ...
[20220829-17:04:03] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20220829-17:04:12] [prefect] Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/clumps_af_300_gene_test_postprocess__2022-08-29--17-04-12_cjnglma_2vzaanq_ttfqonvxt5xyg/jobs/0'
[20220829-17:04:12] [prefect] Overwriting output and aborting job avoidance.
[20220829-17:04:12] [prefect] Localizing inputs...
[20220829-17:04:12] [prefect] Job staged on SLURM controller in: /mnt/nfs/workspace/clumps_af_300_gene_test_postprocess__2022-08-29--17-04-12_cjnglma_2vzaanq_ttfqonvxt5xyg
[20220829-17:04:12] [prefect] Preparing pipeline script
[20220829-17:08:14] [prefect] Finished with status FAILED
[20220829-17:08:14] [prefect] Not all tasks were successful.
[20220829-17:08:39] [prefect] Terminating all jobs ... 
[20

In [20]:
clumpsPrep_results["prep_outdir"]

'/mnt/nfs/workspace/clumps_full_2hrtimeout_100kmax_pre-processing_task__2022-06-16--20-16-15_ohy20dy_rgz4h0q_xurchlelmcp4c/outputs/0/prep_outdir/clumps_preprocess.tar'

### (4) WORKFLOW
using all 3 pre-defined taks

### (5) SUBMIT the job (run the workflow script)

In [ ]:
clumps_analysis.results


In [ ]:
# running the `prep_task` for full set, since set of 100 completed
clumpsPrep = clumps_prep_task()
clumpsPrep_results = clumpsPrep.run(
    maf = clumpsLocalization_results["maf"],
    genome_2bit = clumpsLocalization_results["genome_2bit"],
    fasta = clumpsLocalization_results["fasta"],
    gpmaps = clumpsLocalization_results["gpmaps"]
)
clumpsRun = clumps_run_task()
clumpsRun_results = clumpsRun.run(
    clumps_preprocess = clumpsPrep_results["prep_outdir"],
    prot2pdb_chunks = chunks_list_18,
    pdb_dir = clumpsLocalization_results["pdb_dir"],
    coverage_track = clumpsLocalization_results["coverage_track"],
    coverage_track_index = clumpsLocalization_results["coverage_track_index"],
    genome_2bit = clumpsLocalization_results["genome_2bit"],
    fasta = clumpsLocalization_results["fasta"],
    gpmaps = clumpsLocalization_results["gpmaps"]
)
clumpsPost = clumps_postprocess_task()
clumpsPost_results = clumpsPost.run(
    clumps_preprocess = clumpsPrep_results["prep_outdir"],
    clumps_results = [clumpsRun_results["run_outdir"]],
    cancer_genes = clumpsLocalization_results["cancer_genes"],
    uniprot_map = clumpsLocalization_results["uniprot_map"],
    pdb_dir = clumpsLocalization_results["pdb_dir"]
)

[20220315-16:55:51] [prefect] Available disk storage at /mnt/nfs is small (53 GB remaining)
[20220315-16:55:51] [prefect] Starting Slurm controller ...
[20220315-16:55:51] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20220315-16:55:52] [prefect] Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/clumps_pre-processing_task__2022-03-15--16-55-52_ohy20dy_vtg5ndy_xurchlelmcp4c/jobs/0'
[20220315-16:55:52] [prefect] Overwriting output and aborting job avoidance.
[20220315-16:55:52] [prefect] Localizing inputs...
[20220315-16:55:52] [prefect] Job staged on SLURM controller in: /mnt/nfs/workspace/clumps_pre-processing_task__2022-03-15--16-55-52_ohy20dy_vtg5ndy_xurchlelmcp4c
[20220315-16:55:52] [prefect] Preparing pipeline script
[20220315-17:47:13] [prefect] Finished with status COMPLETED
[20220315-17:47:13] [prefect] Available disk storage at /mnt/nfs is small (52 GB remaining)
[20220315-17:47:13] [prefect] Starting Slurm